# Load files

In [49]:
from langchain_community.document_loaders import TextLoader         
loader=TextLoader("a.txt")     
text_documents=loader.load()
text_documents

[Document(metadata={'source': 'a.txt'}, page_content='good morning everyone my name is Ayush Jindal and currently I am in fourth year')]

# Load and split the text document

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)
documents[:5]

[Document(metadata={'source': 'a.txt'}, page_content='good morning everyone my name is Ayush Jindal and currently I am in fourth year')]

In [39]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")

In [40]:
documents

[Document(metadata={'source': 'a.txt'}, page_content='good morning everyone my name is Ayush Jindal and currently I am in fourth year')]

# Extract text content from Document objects

In [41]:
texts = [doc.page_content for doc in documents] 

# Create embeddings and FAISS vector store


In [42]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vector_store = FAISS.from_texts(texts, embedding=embeddings)
vector_store.save_local("faiss_index")

# Prompt template 

In [43]:
prompt_template = """
    You are an interviewer. Based on the context below, ask a relevant question from the text.

    Context:
    {context}

    Answer:
    """

model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, google_api_key=os.environ["GOOGLE_API_KEY"])
prompt = PromptTemplate(template=prompt_template, input_variables=["context"])
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [44]:
chain

StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    You are an interviewer. Based on the context below, ask a relevant question from the text.\n\n    Context:\n    {context}\n\n    Answer:\n    '), llm=ChatGoogleGenerativeAI(model='models/gemini-pro', google_api_key=SecretStr('**********'), temperature=0.3, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000236F5D171F0>, default_metadata=()), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context')

In [45]:
question = "Can you summarize the main points discussed in the text?"

In [46]:
context = vector_store.similarity_search(question, k=1)  
context_text = context[0].page_content if context else "No relevant context found."
answer = chain.run(input_documents=context, question=question)

In [47]:
answer

'What is your major and what are your career aspirations?'